# **Experiment Demo Notebook**

This notebook has been prepared to allow for the direct running of our experiments.

## **Setup**

To first run the notebooks, we check whether we're currently using Google Colab or not. We can do so by running the following cell:

In [1]:
# Check if using Colab or not
import os
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from getpass import getpass
    from google.colab import drive # Access the drive
    drive.mount('/content/drive')
    pat = ''
    repo_name = 'stefanvasilev/egnn-transformer'
    url = f"https://{pat}@github.com/{repo_name}.git"
    # !git clone --branch nbody_transformer {url}
    !git clone --branch main {url}
    print("\nCurrent Directory:")
    %cd egnn-transformer

Now we can setup the environment and location. If you're not using Colab, then you need to install the environment yourself (which can be done using one of the `.yml` files, which depends on your device).

In [2]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Read the requirements.txt file
    with open('requirements.txt') as f:
        requirements = f.read().splitlines()

    # Check if each requirement is installed, if not, install it
    import pkg_resources
    installed_packages = {pkg.key for pkg in pkg_resources.working_set}
    for requirement in requirements:
        if not any(requirement.split('==')[0] in pkg for pkg in installed_packages):
            !pip install {requirement}

    !pip install datasets


else: # automatically checks if the current directory is 'repo name'
    curdir = Path.cwd()
    print("Current Directory", curdir)
    assert curdir.name == "egnn-transformer" or curdir.parent.name == "egnn-transformer", "Notebook cwd has to be on the project root"
    if curdir.name == "notebooks":
        %cd ..
        print("New Current Directory:", curdir)

Current Directory c:\Users\Gregory Go\.github\egnn-transformer\notebooks
c:\Users\Gregory Go\.github\egnn-transformer
New Current Directory: c:\Users\Gregory Go\.github\egnn-transformer\notebooks


### **Generating the N-body Dataset**

As the N-body dataset is made via simulation, it's possible to create it manually (and should not take long). We can do that by running the following script:

In [8]:
!python ../n_body/dataset/generate_dataset.py --initial_vel 1 --num-train 3000 --length 1000 --length_test 1000 --sufix "small"

python: can't open file '/home/jonas/Desktop/DL2/egnn-transformer/notebooks/generate_dataset.py': [Errno 2] No such file or directory


The data for N-body should now be in `n_body/dataset/data`. Alternatively, you can download the dataset [here](https://drive.google.com/drive/folders/1xfigu6ZJHvw7smx4J_-p8uRryIYGUjK7?usp=sharing).

Note that the data for QM9 is already available from `torch_geometric`, meaning that nothing else needs to be done for it.

## **Experiments**

Now we can begin with the experiments. We first show how to perform those for QM9.

Note that the parameters which can be adjusted are documented in the `README` file.

For the EGNN:

In [4]:
!python ../main_qm9.py

Random seed set as 42


Traceback (most recent call last):
  File "c:\Users\Gregory Go\.github\egnn-transformer\main_qm9.py", line 289, in <module>
    model = get_model(parsed_args)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Gregory Go\.github\egnn-transformer\utils\utils.py", line 150, in get_model
    from models.egnn_jax import EGNN
ImportError: cannot import name 'EGNN' from 'models.egnn_jax' (c:\Users\Gregory Go\.github\egnn-transformer\models\egnn_jax.py)


For the Transformer:

In [ ]:
!python ../transformer_trainer.py

...and now for N-body.

N-body EGNN (for colab it is best to use the colab notebook)

In [3]:
!python ../nbody_egnn_trainer.py --nbody_path ../n_body/dataset/data/

Random seed set as 42
Parameters: 117575
[Epoch  1] Training mse: 0.009367193095386028, Validation mse: 0.008887327276170254
	   (New best performance, saving model...)
Figure(1000x600)
[Epoch  2] Training mse: 0.008097128011286259, Validation mse: 0.007526990957558155
	   (New best performance, saving model...)
[Epoch  3] Training mse: 0.006653978023678064, Validation mse: 0.006075744982808828
	   (New best performance, saving model...)
[Epoch  4] Training mse: 0.005744718946516514, Validation mse: 0.00560001889243722
	   (New best performance, saving model...)
^C
Traceback (most recent call last):
  File "/home/jonas/Desktop/DL2/egnn-transformer/notebooks/../nbody_egnn_trainer.py", line 297, in <module>
    train_model(parsed_args, batch_transform, "test", "assets")
  File "/home/jonas/Desktop/DL2/egnn-transformer/notebooks/../nbody_egnn_trainer.py", line 197, in train_model
    test_loss = eval_fn(test_loader, params)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jonas/

Ablation studies Equivariance

In [7]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "roto_translation" --batch_size 500 --epochs 40

Random seed set as 42
Parameters: 92931
Epoch 1:   0%|                                            | 0/6 [00:00<?, ?it/s]^C


In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "velo_roto_translation" --batch_size 500 --epochs 40

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "translation" --batch_size 500 --epochs 40

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "not_equivariant" --batch_size 500 --epochs 40

Ablation studies transformer architecture

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "velo_roto_translation" --batch_size 500 --epochs 40 --num_edge_encoders 0 --num_node_encoders 2

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "velo_roto_translation" --batch_size 500 --epochs 40 --node_only --num_node_encoders 4

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "velo_roto_translation" --batch_size 500 --epochs 40 --node_only --dim 128

In [ ]:
!python ../nbody_transformer_trainer.py --nbody_path '../n_body/dataset/data/' --equivariance "velo_roto_translation" --batch_size 500 --epochs 40 --dim 128